pip install tensorflow  


In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import imageio
import math
import IPython.display
import PIL

def imshow(data):
    pil_img = PIL.Image.fromarray(data, 'RGB')
    display(pil_img)

tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
patch_size = 64
mini_batch_size = 5
percent_used_for_validation = 0.1
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [3]:
def make_nn():
    #basic model parameters
    L = 9
    kernel_size = 5
    input_channels = 40
    hidden_channels = 100
    output_kernel_size = 21
    in_between_layers = L - 2
    initializer = tf.keras.initializers.GlorotUniform()

    # add layers
    model = keras.Sequential()
    model.add(keras.Input(shape=(patch_size, patch_size, input_channels)))
    for i in range(in_between_layers):
        model.add(keras.layers.ZeroPadding2D(padding=(2, 2)))
        model.add(layers.Conv2D(hidden_channels, kernel_size=kernel_size, kernel_initializer=initializer, strides=1, activation="relu"))
    model.add(keras.layers.ZeroPadding2D(padding=(2, 2)))
    model.add(layers.Conv2D(3, kernel_size=kernel_size, strides=1, activation="relu"))

    # compile model
    opt = keras.optimizers.Adam(learning_rate=10e-5)
    model.compile(loss="mean_absolute_error", optimizer="adam", metrics=['accuracy'])
    return model;

model = make_nn()
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d (ZeroPadding2 (None, 68, 68, 40)        0         
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 64, 100)       100100    
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 68, 68, 100)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 100)       250100    
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 68, 68, 100)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 100)       250100    
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 68, 68, 100)       0

In [4]:
def make_3_channels(image):
    if image.ndim == 2: image = np.dstack((image, image, image))
    image = image[:,:,:3]
    return image

def preprocess_color(color, albedo, epsilon=0.00316):
    return color.astype(np.float32) / (albedo + epsilon)

def postprocess_color(color, albedo, epsilon=0.00316):
    return color * (albedo + epsilon)

def preprocess_color_variance(variance, albedo, epsilon=0.00316):
    return variance / (albedo + epsilon)**2

def calculate_gradient(image):
    y, x, c = image.shape
    dx = image[:, 1:, :] - image[:, :x-1, :]
    dy = image[1:, :, :] - image[:y-1, :, :]
    dx = np.append(dx, np.zeros([y, 1, c]), axis=1)
    dy = np.append(dy, np.zeros([1, x, c]), axis=0)
    grad = np.dstack((dx, dy))
    return grad

def load_highspp(folder_path):
    albedo = make_3_channels(imageio.imread(folder_path + "/albedo.png").astype(np.float32))
    color = make_3_channels(imageio.imread(folder_path + "/color.png").astype(np.float32))
    color = preprocess_color(color, albedo)
    return color

def load_lowspp(folder_path):
    print("loading " + folder_path)

    # albedo buffers
    albedo = make_3_channels(imageio.imread(folder_path + "/albedo.png").astype(np.float32))
    albedo_variance = make_3_channels(imageio.imread(folder_path + "/albedoVariance.png").astype(np.float32))
    albedo_gradient = calculate_gradient(albedo)

    # color buffers
    color = make_3_channels(imageio.imread(folder_path + "/color.png").astype(np.float32))
    color = preprocess_color(color, albedo)
    color_variance = make_3_channels(imageio.imread(folder_path + "/colorVariance.png").astype(np.float32))
    color_variance = preprocess_color_variance(color_variance, albedo)
    color_gradient = calculate_gradient(color)

    # depth buffers
    depth = imageio.imread(folder_path + "/depth.png").astype(np.float32)
    depth = depth.reshape((depth.shape[0], depth.shape[1], 1))
    depth_variance = imageio.imread(folder_path + "/depthVariance.png").astype(np.float32)
    depth = depth.reshape((depth.shape[0], depth.shape[1], 1))
    depth_gradient = calculate_gradient(depth)

    # normal buffers
    normal = make_3_channels(imageio.imread(folder_path + "/normal.png").astype(np.float32))
    normal_variance = make_3_channels(imageio.imread(folder_path + "/normalVariance.png").astype(np.float32))
    normal_gradient = calculate_gradient(normal)

    combined = np.dstack((albedo, albedo_variance, albedo_gradient, color, color_variance, color_gradient, depth, depth_variance, depth_gradient, normal, normal_variance, normal_gradient))
    return combined

In [5]:
# split full res image into patch_size * patch_size images
def get_patches(image):
    x_patches = math.floor(image.shape[0]/patch_size)
    y_patches = math.floor(image.shape[1]/patch_size)
    patches = []
    for x in range(x_patches):
        for y in range(y_patches):
            xstart = x * patch_size
            xend = xstart + patch_size
            ystart = y * patch_size
            yend = ystart + patch_size
            temp = image[xstart:xend, ystart:yend, :]
            patches.append(temp)
        patches.append(image[xstart:xend, image.shape[1]-patch_size:, :])
    return patches

In [6]:
# note to self: use this https://www.tensorflow.org/api_docs/python/tf/data/experimental/sample_from_datasets
def load_x_y_patches():
    x_patches = []
    y_patches = []
    orig = next(os.walk('./pngs'))[1]
    folders = [x[0:x.rindex('16')] for x in orig if '16' in x]

    for x in folders:
        low_spp = load_lowspp('./pngs/' + x + '16')
        high_spp = load_highspp('./pngs/' + x + '4096')
        # residual = high_spp - low_spp[:, :, 12:15]
        x_patches = x_patches + get_patches(low_spp)
        y_patches = y_patches + get_patches(high_spp)
    
    return x_patches, y_patches


In [7]:
x_patches, y_patches = load_x_y_patches()

loading ./pngs/bathroom16
loading ./pngs/bathroom2-16
loading ./pngs/bedroom-16
loading ./pngs/car-16
loading ./pngs/car2-16
loading ./pngs/classroom-16
loading ./pngs/coffee-16
loading ./pngs/cornell-box-16
loading ./pngs/curly-hair-16
loading ./pngs/dining-room-16
loading ./pngs/dragon-16
loading ./pngs/furball-16
loading ./pngs/glass-of-water-16
loading ./pngs/house-16
loading ./pngs/kitchen-16
loading ./pngs/lamp-16
loading ./pngs/living-room-16
loading ./pngs/living-room-2-16
loading ./pngs/living-room-3-16
loading ./pngs/material-testball-16
loading ./pngs/spaceship-16
loading ./pngs/staircase-16
loading ./pngs/staircase2-16
loading ./pngs/teapot-16
loading ./pngs/teapot-full-16
loading ./pngs/veach-ajar-16
loading ./pngs/veach-bidir-16
loading ./pngs/veach-mis-16
loading ./pngs/volumetric-caustic-16


In [8]:
# test = imageio.imread("./pngs/bathroom2-16/color.png")
# test2 = imageio.imread("./pngs/bathroom2-16/albedo.png")

# test3 = imageio.imread("./pngs/bathroom2-4096/color.png")
# test4 = imageio.imread("./pngs/bathroom2-16/albedo.png")


# print("test")
# print(test)
# test_patch = x_patches[500]
# test3 = test_patch[:,:,12:15]
# test4 = (test3 * 100.0).astype(np.uint8)
# test3 = postprocess_color(test3, test_patch[:,:,:3]).astype(np.uint8)

# print("test3" + str(test3.shape))
# print(test3.dtype)

# test3 = test3/np.linalg.norm(test3)

# imshow(test3-test)
# x_patches[0][:,:,:3]

In [9]:
def create_tf_dataset(x_patches, y_patches):
    validate_amount = math.floor(len(x_patches) * percent_used_for_validation)
    rec_count = len(x_patches)
    ds = tf.data.Dataset.from_tensor_slices((x_patches, y_patches))
    ds = ds.shuffle(buffer_size=400, reshuffle_each_iteration=True)
    train = ds.skip(validate_amount).batch(mini_batch_size)
    validate = ds.take(validate_amount).batch(mini_batch_size)
    return train, validate

In [10]:
# train, validate = prepare_training_data()
train, validate = create_tf_dataset(x_patches, y_patches)

In [12]:
print("Fit model on training data")
history_cb = tf.keras.callbacks.CSVLogger('./training_1/log.csv', separator=",", append=False)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
history = model.fit(
    train,
    validation_data=validate,
    epochs=10000,
    callbacks=[cp_callback, history_cb]
)

Fit model on training data
Epoch 1/10000
1260/1260 [==============================] - 144s 104ms/step - loss: 1409.8458 - accuracy: 0.7540 - val_loss: 103.7638 - val_accuracy: 0.6480

Epoch 00001: saving model to training_1\cp.ckpt
Epoch 2/10000
 318/1260 [======>.......................] - ETA: 1:04 - loss: 16.4363 - accuracy: 0.7953

KeyboardInterrupt: 